In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import math
from tensorflow.keras.layers import Concatenate, Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from functions import feature_extraction
import warnings
# from pandas.core.common import SettingWithCopyWarning

# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [21]:
AEP = pd.read_csv('AEP_hourly.csv')
AEP = AEP.rename(columns={'AEP_MW': 'Load'})
num_days_ago = 21
num_hours_forecasting = 24
features, data = feature_extraction(AEP, diff_in_hour=1, num_days_ago=num_days_ago, num_hours_forecasting=num_hours_forecasting,  test_split = 0.1, validation_split = 0.2)

/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/functions.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i * diff_in_hour}_hrs_ago'] = df['Load'].shift(i)
/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/functions.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i * diff_in_hour}_hrs_ago'] = df['Load'].shift(i)
/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/fu

/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/functions.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i * diff_in_hour}_hrs_ago'] = df['Load'].shift(i)
/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/functions.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i * diff_in_hour}_hrs_ago'] = df['Load'].shift(i)
/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/fu

/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/functions.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i * diff_in_hour}_hrs_ago'] = df['Load'].shift(i)
/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/functions.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i * diff_in_hour}_hrs_ago'] = df['Load'].shift(i)
/Users/eliorsegev/Library/Mobile Documents/com~apple~CloudDocs/Studying/Uni/Master/papers/ps_ctrl/fu

In [22]:
# data preparing

x_train_seq = data['x_train_seq'].reshape(data['x_train_seq'].shape[0], data['x_train_seq'].shape[1],1)
y_train = data['y_train'].reshape(data['y_train'].shape[0], data['y_train'].shape[1],1)
x_val_seq = data['x_val_seq'].reshape(data['x_val_seq'].shape[0], data['x_val_seq'].shape[1],1)
y_val = data['y_val']

In [23]:
# an LSTM Model:

In [24]:
# Define parameters
input_seq_len = num_days_ago * 24
output_seq_len = num_hours_forecasting
n_features = 1

# Define the layers
encoder_inputs = Input(shape=(input_seq_len, n_features))
encoder = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, which will use `encoder_states` as initial state.
decoder_lstm = LSTM(64, return_sequences=True)
decoder_outputs = decoder_lstm(RepeatVector(output_seq_len)(encoder_outputs), initial_state=encoder_states)

# Apply a dense layer to the every temporal slice of an input
decoder_dense = TimeDistributed(Dense(n_features))
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(encoder_inputs, decoder_outputs)

model.compile(optimizer='adam', loss='mse')

history = model.fit(x_train_seq, y_train, batch_size=256, epochs=7, validation_data = (x_val_seq, y_val))



Epoch 1/7
 57/330 [====>.........................] - ETA: 2:30 - loss: 0.8072

KeyboardInterrupt: 

In [ ]:
plt.plot(range(1, len(history.history['loss'])+1),history.history['loss'], label='train loss')
plt.plot(range(1, len(history.history['val_loss'])+1), history.history['val_loss'], label='validation loss')
plt.scatter(range(1, len(history.history['loss'])+1),history.history['loss'])
plt.scatter(range(1, len(history.history['val_loss'])+1), history.history['val_loss'])
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_hat = np.array(model.predict(x_val_seq))
y_hat = y_hat*data['std_value_load'] + data['mean_value_load']
y = y_val*data['std_value_load'] + data['mean_value_load']

In [ ]:
index = 2500

print(f'y mean: {y.mean()}, y std: {y.std()}')
print(f'y_hat mean: {y_hat.mean()}, y_hat std: {y_hat.std()}')

y_plot_hat = y_hat[index,:]
y_plot = y[index,:]

plt.plot(range(len(y_plot_hat)), y_plot_hat, label='predicted load')
plt.plot(range(len(y_plot)), y_plot, label='real load')
plt.scatter(range(len(y_plot_hat)), y_plot_hat)
plt.scatter(range(len(y_plot)), y_plot)
plt.title('prediction')
plt.xlabel('Time')
plt.ylabel('Normalized load')
plt.legend()
plt.show()